### TTN generation sandbox

New method:
Instead of using pyGeo, we now just use regular pyTorch.

To output the graph, we'll interpret an output vector as probabilities in a graph.
Post-processing probably required for:
- ensuring graph is connected
- ensuring graph is tree-like

In [7]:
# %automagic

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [21]:


input_dim = 2
output_dim = 2

# Define the model class
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        # First hidden layer
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        # Second hidden layer
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        # Output layer
        self.fc3 = nn.Linear(hidden_dim, output_dim * output_dim)

    def forward(self, x):
        # Pass through hidden layers with activation
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        # Output layer without activation (linear regression)
        return self.fc3(x).view(x.shape[0], output_dim, output_dim)

# Sample training data (modify this based on your actual data)
input_data = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
output_data = torch.tensor([ [[3.0, 4.0],[3.0, 4.0]],
                             [[7.0, 8.0], [7.0, 8.0]],
                             [[11.0, 12.0],[11.0, 12.0]]  ]  )

# Create a dataset and dataloader
dataset = TensorDataset(input_data, output_data)
dataloader = DataLoader(dataset, batch_size=2)  # Adjust batch_size as needed

# Define model, loss function, and optimizer
model = MLP(input_data.shape[1], 16, output_data.shape[1])  # Adjust hidden layer size
loss_fn = nn.MSELoss()  # Mean Squared Error loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

# Train the model
for epoch in range(100):  # Adjust number of epochs for better training
    for data, target in dataloader:
        optimizer.zero_grad()
        prediction = model(data)
        loss = loss_fn(prediction, target)
        loss.backward()
        optimizer.step()


# Example usage: predict output for a new input
new_input = torch.tensor([[7.0, 8.0]])
predicted_output = model(new_input)
print(f"Predicted output for new input:\n {predicted_output}")

Predicted output for new input:
 tensor([[[14.9579, 16.0895],
         [14.9429, 16.0371]]], grad_fn=<ViewBackward0>)
